## Data loading and cleaning

In [16]:
# Useful starting lines

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import datetime


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from proj1_helpers import *
DATA_TRAIN_PATH = '/Users/akhileshgotmare/Desktop/Git_Junta/data-ml-course-project1/train.csv' # TODO: download train data and supply path here 
y, X, ids = load_csv_data(DATA_TRAIN_PATH)

In [18]:
from helpers import *

def standardize_badFeatures(X):
    
    # Function that calculate the mean and std of bad features without elements equal to -999
    # Then, it replaces -999 values by zeros, zeros won't influence the train of the model... 
    mean_x = np.zeros((X.shape[1],))
    std_x = np.zeros((X.shape[1],))
    for d in range(X.shape[1]):
        idx = np.where(X[:,d] == -999)[0]
        mean_x[d] = np.mean(np.delete(X[:,d], (idx)))
        std_x[d] = np.std(np.delete(X[:,d], (idx)))
        X[:,d] = (X[:,d]-mean_x[d])/std_x[d]
        X[idx,d] = 0
    return X, mean_x, std_x


def clean_data(X):

    # find indices of features that have at least one value -999, we call them "bad" features
    idx_badFeatures = []
    for d in range(X.shape[1]):
        if sum(X[:,d] == -999) > 0:
            idx_badFeatures.append(d)

    # separate "good" and "bad" features
    X_badFeatures = X[:,idx_badFeatures]
    X_goodFeatures = np.delete(X,(idx_badFeatures), axis=1)

    # Standardize it differently (see : standardize_badFeatures(X))
    tX, mean_x, std_x = standardize(X_goodFeatures)
    tX2, mean_x2, std_x2 = standardize_badFeatures(X_badFeatures)

    # comment the 3 next lines if you want to work only with "good" features
    tX = np.hstack((tX, tX2))
    mean_x = np.hstack((mean_x, mean_x2))
    std_x = np.hstack((std_x, std_x2))
    
    return tX, mean_x, std_x

# Now tX already has ones in the first column...
tX, mean_x, std_x = clean_data(X)
X=tX

In [19]:
y[np.where(y == -1)[0]] = 0

## Penalized logistic regression 

In [96]:
res

array([ 33.46867866,  20.70679819,   3.17998352, ...,   3.24552249,
         3.83250566,   3.94738714])

In [100]:
def sigma(a):
    
    sig=np.zeros([a.shape[0]],)
    
    for i in range(a.shape[0]):
        if a[i]>100:
            sig[i] = 1
        elif a[i]<-100:
            sig[i] = 0
        else:
            sig[i]=np.exp(a[i])/(1+ np.exp(a[i]))
        
                
    
    
    
    return sig

In [106]:
i=0


m=15
n=15
P = np.zeros([m+1,n+1])
 
max_degree=m-1
gamma = 0.01

import random
import myFunctions as my
ws=[]

megacounter = 0
for degree in range(m):
    j=0
    for lamb in np.logspace(-5, 2, n):
        print(i,j)
        print(megacounter)
        megacounter+=1
        max_iters = 600
        
        nums = [x for x in range(X.shape[0])]
        random.shuffle(nums)

        #row_r1 = a[1, :] 
        stop=int(X.shape[0]/10)
        
        X_test = X[nums[0:stop],:]
        X_train = X[nums[stop+1:-1],:]
        
        y_test = y[nums[0:stop]]
        y_train = y[nums[stop+1:-1]]
        
        
        X_test_poly = my.build_poly(X_test, degree)
        X_train_poly = my.build_poly(X_train, degree)
        
        
        initial_w=np.random.random([X_train_poly.shape[1]],)
        w=initial_w
        
        for n_iter in range(max_iters):
            
            #print(i,j)
            #print(n_iter)
            gradient = X_train_poly.T.dot(sigma(X_train_poly.dot(w)) - y_train) + 2*lamb*w
            w = w - gamma*gradient
        
        y_test_predict = X_test_poly.dot(w) 
        y_test_predict[np.where(y_test_predict >= 0.5)[0]] = 1
        y_test_predict[np.where(y_test_predict <= 0.5)[0]] = 0
        
        error = len(np.nonzero(y_test_predict-y_test)[0])/len(y_test)
        performance = 1 - error
        
        P[i,j] = performance
        
        ws.append(np.copy(w))
            
        j+= 1
    i+=1


0 0
0
0 1
1
0 2
2
0 3
3
0 4
4
0 5
5
0 6
6
0 7
7
0 8
8
0 9
9
0 10
10
0 11
11
0 12
12
0 13
13
0 14
14
1 0
15
1 1
16
1 2
17
1 3
18
1 4
19
1 5
20
1 6
21
1 7
22
1 8
23
1 9
24
1 10
25
1 11
26
1 12
27
1 13
28
1 14
29
2 0
30
2 1
31
2 2
32
2 3
33
2 4
34
2 5
35
2 6
36
2 7
37
2 8
38
2 9
39
2 10
40
2 11
41
2 12
42
2 13
43
2 14
44
3 0
45
3 1
46
3 2
47
3 3
48
3 4
49
3 5
50
3 6
51
3 7
52
3 8
53
3 9
54
3 10
55
3 11
56
3 12
57
3 13
58
3 14
59
4 0
60
4 1
61
4 2
62
4 3
63
4 4
64
4 5
65
4 6
66
4 7
67
4 8
68
4 9
69
4 10
70
4 11
71
4 12
72
4 13
73
4 14
74
5 0
75
5 1
76
5 2
77
5 3
78
5 4
79
5 5
80
5 6
81
5 7
82
5 8
83
5 9
84
5 10
85
5 11
86
5 12
87
5 13
88
5 14
89
6 0
90
6 1
91
6 2
92
6 3
93
6 4
94
6 5
95
6 6
96
6 7
97
6 8
98
6 9
99
6 10
100
6 11
101
6 12
102
6 13
103
6 14
104
7 0
105
7 1
106
7 2
107
7 3
108
7 4
109
7 5
110
7 6
111
7 7
112
7 8
113
7 9
114
7 10
115
7 11
116
7 12
117
7 13
118
7 14
119
8 0
120
8 1
121
8 2
122
8 3
123
8 4
124
8 5
125
8 6
126
8 7
127
8 8
128
8 9
129
8 10
130
8 11
131
8 12
132
8 1

In [108]:
P

array([[ 0.65304,  0.6566 ,  0.34236,  0.66288,  0.33988,  0.33584,
         0.34116,  0.3434 ,  0.66044,  0.66128,  0.65688,  0.65544,
         0.34344,  0.34336,  0.34392,  0.     ],
       [ 0.69172,  0.69468,  0.69956,  0.6912 ,  0.68876,  0.6948 ,
         0.6924 ,  0.69528,  0.69184,  0.70376,  0.71088,  0.71408,
         0.67224,  0.6502 ,  0.62964,  0.     ],
       [ 0.73744,  0.74592,  0.74264,  0.74852,  0.74648,  0.729  ,
         0.73472,  0.72304,  0.71264,  0.7364 ,  0.67832,  0.662  ,
         0.657  ,  0.35728,  0.34356,  0.     ],
       [ 0.74196,  0.74004,  0.761  ,  0.74344,  0.67628,  0.72332,
         0.7746 ,  0.75244,  0.72612,  0.7576 ,  0.66948,  0.64476,
         0.71228,  0.67712,  0.5292 ,  0.     ],
       [ 0.75296,  0.74544,  0.74604,  0.73184,  0.73256,  0.74648,
         0.75976,  0.7662 ,  0.60868,  0.72108,  0.68916,  0.51392,
         0.52484,  0.57804,  0.34384,  0.     ],
       [ 0.57868,  0.58016,  0.58072,  0.57736,  0.581  ,  0.6834 ,
       

In [110]:
P[3,6]

0.77459999999999996

In [114]:
ws[51].shape

(91,)

In [115]:
for lamb in np.logspace(-5, 2, n):
    print(lamb)

1e-05
3.16227766017e-05
0.0001
0.000316227766017
0.001
0.00316227766017
0.01
0.0316227766017
0.1
0.316227766017
1.0
3.16227766017
10.0
31.6227766017
100.0


## Generate predictions with ws[51]

In [ ]:
DATA_TEST_PATH = '/Users/akhileshgotmare/Desktop/Git_Junta/data-ml-course-project1/test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
#X_test = np.delete(X_test, del_features, axis=1)
tX_test, mean_tX_test, std_tX_test = standardize(X_test)
print(tX_test.shape)

In [119]:
ws[51]

array([  2.38275805e+03,  -4.03215163e+03,  -3.00152050e+03,
         1.63606080e+03,   3.24782312e+02,  -3.81922681e+03,
         2.11079515e+03,  -1.08840782e+03,   8.09333759e+02,
         1.14798053e+03,  -8.92103226e+01,   4.71066940e+01,
         3.32241297e+03,   1.35583755e+02,   9.60290265e+01,
        -2.61186380e+03,  -7.90142572e+00,   7.86164376e+02,
         8.14881300e+01,   1.48100934e+03,   3.81643049e+03,
         1.08361236e+03,   3.93780904e+03,  -1.30023583e+03,
         1.70400500e+03,   5.32108735e+02,  -1.08884659e+02,
        -1.39715845e+02,   3.93721606e+03,   3.67055747e+01,
        -8.98326003e+01,  -4.67349093e+02,  -5.62990755e+03,
         4.39407424e+03,  -8.52161695e+03,  -1.02325394e+04,
         1.20379556e+03,  -9.15922076e+01,   1.88449419e+03,
         3.05123229e+03,  -4.35520884e+02,   1.78417537e+02,
         1.39180745e+03,  -7.01567136e+02,   2.28436764e+02,
        -2.57262428e+03,   1.70998373e+02,  -5.20677363e+01,
        -2.80558466e+02,